Şu ana kadar Pandas Serie ve DataFrame nesnelerinde bir ve iki boyutlu dizileri gördük, üzerinde çalıştık. Ancak bize iki boyuttan fazlası gerekebilir.

Pandas üç ve dört boyutlu veriler için Panel ve Panel4D nesneleri bulundurur. Ancak 2 boyutun üstüne çıkmaktansa index hiyerarşisi (indexlerin altında indexler) oluşturmak daha yaygın bir çözümdür.

Bu bölümde MultiIndex yapısını ve üzerinde işlem yapmayı öğreneceğiz.

## Serie'lerde Çoklu Index

Öncelikle iki boyutlu veriyi nasıl tek boyutlu Serie'ye çevirebileceğimize bakalım.

Aklınıza bunun için farklı yöntemler gelmiş olabilir ama Pandas MultiIndex veri yapısı bu iş için üretilmiştir ve işinizi kolaylaştırır. 

Aşağıda üç tane şehrin 2010 ve 2015 yılındaki nüfuslarını tek indexte tutacağız:

In [1]:
import numpy as np
import pandas as pd

index = [('Ankara', 2010),    ('Ankara', 2015),
         ('Muş', 2010),       ('Muş', 2015),
         ('Gümüşhane', 2010), ('Gümüşhane', 2015)]
nufuslar = [4771716, 5270575,
            406886,  408728,
            129618,  151449]

ind = pd.MultiIndex.from_tuples(index)
nufus = pd.Series(nufuslar, index=ind)
nufus

Ankara     2010    4771716
           2015    5270575
Muş        2010     406886
           2015     408728
Gümüşhane  2010     129618
           2015     151449
dtype: int64

Yukarıda **tuple**'larla indexlerimizi oluşturup `MultiIndex.from_tuples` fonksiyonu ile `MultiIndex` veri tipine dönüştürdük. Artık verimizi iki boyutlu dizi gibi kullanabiliriz.

In [2]:
nufus["Muş"][2015]

408728

In [3]:
nufus[:, 2010]

Ankara       4771716
Muş           406886
Gümüşhane     129618
dtype: int64

### Çok Boyutlu MultiIndex

Bu veriyi DataFrame kullanarak da depolayabilirdik. Elbette Pandas'ı üretenler de bunu farkında ve bunun için bir fonksiyon yazmışlar. `unstack` fonksiyonu ile çoklu indexlenmiş veriyi kolayca DataFrame'e çevirebilirsiniz.

In [4]:
nufus.unstack()

,2010,2015
Ankara,4771716,5270575
Gümüşhane,129618,151449
Muş,406886,408728


Beklenildiği üzere `stack` fonksiyonu da bunun tam tersi, DataFrame'i MultiIndex yapar.

## MultiIndex Üretme

Bir MultIndex üretmenin en düz yolu, index olarak iki boyutlu dizi vermektir:

In [5]:
pd.DataFrame(np.random.randint(10, size=(4, 2)),
             index=[['A', 'A', 'B', 'B'], [1, 2, 1, 2]],
             columns=['veri1', 'veri2'])

veri1  veri2
A 1      6      5
  2      1      7
B 1      2      7
  2      5      0

Diğer bir yöntem anahtarları tuple olan sözlükler kullanmaktır.

In [6]:
sozlk = {('Ankara', 2000):   4771716,
        ('Ankara', 2010):    5270575,
        ('Muş', 2000):       406886,
        ('Muş', 2010):       408728,
        ('Gümüşhane', 2000): 129618,
        ('Gümüşhane', 2010): 151449}
pd.Series(sozlk)

Ankara     2000    4771716
           2010    5270575
Muş        2000     406886
           2010     408728
Gümüşhane  2000     129618
           2010     151449
dtype: int64

### MultıIndex Üreten Fonksiyonlar

pd.MultiIndex sınıfı altında MultiIndex üretmek için bir sürü metod bulunur. Bunlardan biri az önce gördüğümüz `from_tuples()` idi.

In [7]:
pd.MultiIndex.from_tuples([('A', 1), ('A', 2), ('B', 1), ('B', 2)])

MultiIndex([('A', 1),
            ('A', 2),
            ('B', 1),
            ('B', 2)],
           )

Bu fonksiyonlardan bir diğeri de `from_arrays()`.

In [8]:
pd.MultiIndex.from_arrays([['A', 'A', 'B', 'B'], [1, 2, 1, 2]])

MultiIndex([('A', 1),
            ('A', 2),
            ('B', 1),
            ('B', 2)],
           )

`from_product()` fonksiyonuyla iki dizinin kartezyen çarpımları ile de MultiIndex üretilebilir.

In [9]:
pd.MultiIndex.from_product([['A', 'B'], [1, 2]])

MultiIndex([('A', 1),
            ('A', 2),
            ('B', 1),
            ('B', 2)],
           )

Direkt `MultiIndex`i kullanarak da MultiIndex tanımlayabilirsiniz. Parametre olarak `levels`e index isimlerini, `codes`e de `levels`e yazılan indexlerin indexlerini alır.

In [10]:
pd.MultiIndex(levels=[['A', 'B'], [1, 2]],
              codes=[[0, 0, 1, 1], [0, 1, 0, 1]])

MultiIndex([('A', 1),
            ('A', 2),
            ('B', 1),
            ('B', 2)],
           )

#### MultiIndex Index Adları

Daha komplike verilerde indexleri adlandırmak işe yarayabilir. Index'leri aşağıdaki gibi adlandırabiliriz.

In [11]:
nufus.index.names = ['Yer', 'Yıl']
nufus

Yer        Yıl 
Ankara     2010    4771716
           2015    5270575
Muş        2010     406886
           2015     408728
Gümüşhane  2010     129618
           2015     151449
dtype: int64

#### Sütunlarda MultiIndex

Satırlarda olduğu gibi sütunlarda da MultiIndex yapısı kullanılabilir.

Aşağıdaki örnekte rastgele kişilerin boy ve kilolarını 2015 ve 2020 yıllarında şubat ve eylül aylarında gösteren DataFrame oluşturuyorum.

In [12]:
# index ve sütunlar
satir = pd.MultiIndex.from_product([[2015, 2020], ["Şubat", "Eylül"]], names=['Yıl', 'Dönem'])
sutun = pd.MultiIndex.from_product([['Faruk', 'Cemil', 'Tarık'], ['Kilo', 'Tansiyon']],
names=['Kişi', 'Ölçülen'])

# rastgele veriler
veri = np.array([[np.random.randint(55, 100), np.random.randint(70, 130),
                  np.random.randint(55, 100), np.random.randint(70, 130),
                  np.random.randint(55, 100), np.random.randint(70, 130)]
                for i in range(4)])

# ve Dataframe
olcumler = pd.DataFrame(veri, index=satir, columns=sutun)
olcumler

Kişi       Faruk          Cemil          Tarık         
Ölçülen     Kilo Tansiyon  Kilo Tansiyon  Kilo Tansiyon
Yıl  Dönem                                             
2015 Şubat    77      103    77      119    66      114
     Eylül    92       70    79       76    84      106
2020 Şubat    70       75    88      107    96       83
     Eylül    93      119    55      128    93       79

Normalde dört boyutlu bir veri olacakken MultiIndex sayesinde iki boyutlu tabloda görebiliyoruz, verilere de bildiğimiz yöntemlerle ulaşabiliyoruz.

In [13]:
olcumler['Cemil']

Ölçülen     Kilo  Tansiyon
Yıl  Dönem                
2015 Şubat    77       119
     Eylül    79        76
2020 Şubat    88       107
     Eylül    55       128

In [14]:
olcumler['Faruk', 'Kilo'][2015]

Dönem
Şubat    77
Eylül    92
Name: (Faruk, Kilo), dtype: int64

## MultiIndex Seçme ve Bölümleme

MultiIndex yapısında Indexing ve bölümleme (slicing) işlem yapmak için çok boyutlu dizilermiş gibi düşünebilirsiniz. Önce Serie'ler, sonra da DataFrame'ler üzerinde örneklerimize bakalım.

### MultiIndex Serie

Veri seçme örnekleri için daha önceden kullandığımız `nufus` verisini kullanacağız.

In [15]:
nufus

Yer        Yıl 
Ankara     2010    4771716
           2015    5270575
Muş        2010     406886
           2015     408728
Gümüşhane  2010     129618
           2015     151449
dtype: int64

Bilindiği gibi elemanlara indexleri belirterek ulaşabiliriz.

In [16]:
nufus["Ankara", 2010]

4771716

Kısmi seçim (Partial Indexing) işleminde indexlerden sadece biri verilerek bir Serie'ye ulaşılır.

In [17]:
nufus['Gümüşhane']

Yıl
2010    129618
2015    151449
dtype: int64

Index sıralı ise (sıralı indexleri az sonra göreceğiz) `nufus.loc['Muş':'Gümüşhane']` şeklinde bölümleme (slicing) işlemi de yapılabilir.

İlk indexe boş bölümleme işareti koyarak ikincil index üzerinde işlem yapılır.

In [18]:
nufus[:, 2010]

Yer
Ankara       4771716
Muş           406886
Gümüşhane     129618
dtype: int64

[Boolean maskeleme](Numpy6) de kullanılabilir.

In [19]:
nufus[1000000 < nufus]

Yer     Yıl 
Ankara  2010    4771716
        2015    5270575
dtype: int64

[Fancy Indexing](#numpy7) de mümkün.

In [20]:
nufus[['Ankara', 'Gümüşhane']]

Yer        Yıl 
Ankara     2010    4771716
           2015    5270575
Gümüşhane  2010     129618
           2015     151449
dtype: int64

### MultiIndex DataFrame

MultiIndex kullanılmış DataFrame de Serie'ler ile benzer şekilde kontrol edilir. Örnekler için `olcumler` verisini kullanabiliriz.

In [21]:
olcumler

Kişi       Faruk          Cemil          Tarık         
Ölçülen     Kilo Tansiyon  Kilo Tansiyon  Kilo Tansiyon
Yıl  Dönem                                             
2015 Şubat    77      103    77      119    66      114
     Eylül    92       70    79       76    84      106
2020 Şubat    70       75    88      107    96       83
     Eylül    93      119    55      128    93       79

İlk yazdığımız indexte sütun adı yazıldığını hatırlayın.

In [22]:
olcumler['Tarık', 'Kilo']

Yıl   Dönem
2015  Şubat    66
      Eylül    84
2020  Şubat    96
      Eylül    93
Name: (Tarık, Kilo), dtype: int64

Ayrıca [Veri Seçme ve Indexleme](pandas4#Index-Tanımlayıcılar:-loc-ve-iloc) bölümünde öğrendiğimiz `loc` ve `iloc` indexleyicilerini kullanabiliriz. Örneğin:

In [23]:
olcumler.iloc[:2, :2] # konum olarak ikinci (iki hariç) sütun ve satırlar (kapalı index)

Kişi       Faruk         
Ölçülen     Kilo Tansiyon
Yıl  Dönem               
2015 Şubat    77      103
     Eylül    92       70

`loc` ve `iloc` index hiyerarşisi içinde seçim yapacağında indexlerden oluşan tuple alır.

In [24]:
olcumler.loc[:, ('Cemil', 'Tansiyon')]

Yıl   Dönem
2015  Şubat    119
      Eylül     76
2020  Şubat    107
      Eylül    128
Name: (Cemil, Tansiyon), dtype: int64

Bu index tuple'ları içinde bölümleme yapılmaz. Yani `olcumler.loc[:, (:, 'Tansiyon')]` hata verir.

Bu bölümleme işlemini Python ile gelen `slice()` fonksiyonu ile yapabilirsiniz ama daha iyi yolu Pandas'ın `IndexSlice` nesnesini kullanmaktır.

In [25]:
idx = pd.IndexSlice
olcumler.loc[:, idx[:, 'Tansiyon']]

Kişi          Faruk    Cemil    Tarık
Ölçülen    Tansiyon Tansiyon Tansiyon
Yıl  Dönem                           
2015 Şubat      103      119      114
     Eylül       70       76      106
2020 Şubat       75      107       83
     Eylül      119      128       79

### Çoklu Index'in Yapısını Değiştirme

Çoklu Indexlenmiş dizilerle çalışmak için veriyi nasıl çevirip aktaracağınızı bilmeniz çok önemlidir. Veri kaybetmeden çevirmenin birçok yolu var. Bunlardan `stack()` ve `unstack()` fonksiyonlarını az önce gördük. Şimdi bu yöntemleri inceleyeceğiz.??

### Sıralı ve Sıralanmamış Index

Eğer Index sıralı değilse MultiIndex index sırasının önemli olduğu, bölümleme gibi, işlemlerde hata verir. Anlamak için sıralanmamış indexlere sahip şu diziye bakalım.

In [26]:
index = pd.MultiIndex.from_product([['a', 'c', 'b'], [1, 2]])
index
veri = pd.Series(np.random.rand(6), index=index)
veri.index.names = ['harf', 'sayı']
veri

harf  sayı
a     1       0.898970
      2       0.997711
c     1       0.544904
      2       0.182005
b     1       0.127931
      2       0.131567
dtype: float64

`veri['a':'b']` şeklinde bölümleme işlemi yaparsak hata verdiğiniz görürsünüz. Çünkü görüntüde öyle olsa da `"a"` ilk, `"b"` sonuncu index değildir.

Bölümleme ve benzer işlemler için indexlerin sıralı olması gerekir. Indexleri sıralamak için Pandas `sort_index()` ve `sortlevel()` gibi birtakım fonksiyonlar bulundurur. Biz kolay olduğu için `sort_index()` fonksiyonunu kullanacağız. 

In [27]:
veri = veri.sort_index()
veri

harf  sayı
a     1       0.898970
      2       0.997711
b     1       0.127931
      2       0.131567
c     1       0.544904
      2       0.182005
dtype: float64

Indexler sıralandıktan sonra bölümleme ve diğer işlemleri yapabilirsiniz.

In [28]:
veri['a':'b']

harf  sayı
a     1       0.898970
      2       0.997711
b     1       0.127931
      2       0.131567
dtype: float64

### stack() ve unstack()

Daha önce de gördüğümüz üzere, `unstack()` fonksiyonu MultiIndex yapısını diziye çevirir. `stack()` ise tam tersini yapar.

### Indexleri Ayarlama

`reset_index()` fonksiyonu, Index adlarını sütuna çevirerek Serie'yi DataFrame'e dönüştürür. Index olmadığından adı olmayan veriye de ismi `name` parametresi ile verebilirsiniz.

In [29]:
nufus

Yer        Yıl 
Ankara     2010    4771716
           2015    5270575
Muş        2010     406886
           2015     408728
Gümüşhane  2010     129618
           2015     151449
dtype: int64

In [30]:
nufus_df = nufus.reset_index(name='Nüfus')
nufus_df

,Yer,Yıl,Nüfus
0,Ankara,2010,4771716
1,Ankara,2015,5270575
2,Muş,2010,406886
3,Muş,2015,408728
4,Gümüşhane,2010,129618
5,Gümüşhane,2015,151449


Gerçek hayatta bulacağınız veriler yukarıdakine benzer olacak. `set_index()` fonksiyonu da DataFrame'leri MultiIndex DataFrame yapısına çevirir. Parametre olarak liste halinde index'e çevirmek istediğiniz sütun adlarını alır.

In [31]:
nufus_df.set_index(['Yer', 'Yıl'])

Nüfus
Yer       Yıl          
Ankara    2010  4771716
          2015  5270575
Muş       2010   406886
          2015   408728
Gümüşhane 2010   129618
          2015   151449

## MultiIndex Yapısında Genel Veriler

Daha önce de gördüğümüz üzere Pandas'ın `mean()`, `sum()` ve `max()` gibi veri hakkında genel bilgi almamızı sağlayan fonksiyonları vardı. MultiIndex kullanan verilerde bu fonksiyonlar `level` adında parametre alır. `level` parametresi bilgilerin hangi seviye düzeyinde işleneceğini belirtir.

In [32]:
olcumler

Kişi       Faruk          Cemil          Tarık         
Ölçülen     Kilo Tansiyon  Kilo Tansiyon  Kilo Tansiyon
Yıl  Dönem                                             
2015 Şubat    77      103    77      119    66      114
     Eylül    92       70    79       76    84      106
2020 Şubat    70       75    88      107    96       83
     Eylül    93      119    55      128    93       79

In [33]:
olcumler.max()

Kişi   Ölçülen 
Faruk  Kilo         93
       Tansiyon    119
Cemil  Kilo         88
       Tansiyon    128
Tarık  Kilo         96
       Tansiyon    114
dtype: int64

In [34]:
olcumler.max(level='Yıl')

Kişi    Faruk          Cemil          Tarık         
Ölçülen  Kilo Tansiyon  Kilo Tansiyon  Kilo Tansiyon
Yıl                                                 
2015       92      103    79      119    84      114
2020       93      119    88      128    96       83

`axis` parametresi 0:index, 1:sütun üzerinde işlem yapar.

In [35]:
olcumler.max(axis=1, level='Ölçülen')

Ölçülen     Kilo  Tansiyon
Yıl  Dönem                
2015 Şubat    77       119
     Eylül    92       106
2020 Şubat    96       107
     Eylül    93       128

## Panel Veri Türü

Pandas'ta henüz bahsetmediğimiz iki veri yapısı daha var. Bunlar, üç ve dört boyutlu diziye de benzetebileceğimiz `pd.Panel` ve `pd.Panel4D` yapılarıdır. Bu yapılarda da veri seçimi, bölümlemesi ve diğer işlemler benzer şekilde yapılır.

MultiIndex yapısı, çok boyutlu dizileri daha kolay ve kullanışlı ifade eder. Ayrıca dizi boyutu çoğaldıkça gerçek hayattaki verilerden de uzaklaşır. Elbette Panel ve Panel4D yapılarının gerekli olduğu durumlar olabilir. Ama MultiIndex kadar çok kullanılmaz.